# Flame Speed with Sensitivity Analysis

### Import Modules

In [1]:
from __future__ import print_function
from __future__ import division

import cantera as ct
import numpy as np

print("Running Cantera Version: " + str(ct.__version__))

Running Cantera Version: 2.5.1


In [2]:
# Import plotting modules and define plotting preference
%matplotlib notebook
import matplotlib.pylab as plt

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.figsize'] = (8,6)

# Get the best of both ggplot and seaborn
plt.style.use('ggplot')
plt.style.use('seaborn-deep')

plt.rcParams['figure.autolayout'] = True

# Import Pandas for DataFrames
import pandas as pd

### Define the reactant conditions, gas mixture and kinetic mechanism associated with the gas

## Inlet Temperature in Kelvin and Inlet Pressure in Pascals
#In this case we are setting the inlet T and P to room temperature conditions
To = 298
Po = 100000

#Define the gas-mixutre and kinetics
#In this case, we are choosing a GRI3.0 gas
gas = ct.Solution('c2h6/cantera/chem.cti')
i = 0; list = []
while i < .0525: 
    list.append(i)
    i += 0.0025
for x in list: 
    dict = {':x}
    y = 'C2H6(1):%(mole)d, O2(2):(1-%(mole)d)*.21, N2:(1-%(mole)d)*0.79' %dict
    gas.set_equivalence_ratio(1.0, y,'O2(2):0.21, N2:0.79') 
    gas.TP = To, Po
    width = 0.014
    flame = ct.FreeFlame(gas, width=width)
    flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1)
    loglevel = 1
    flame.solve(loglevel=loglevel, auto=True)
# Create a stoichiometric C2H6/Air premixed mixture 
#gas.set_equivalence_ratio(1.0, 'C2H6(1):1.0','O2(2):0.21, N2:0.79')
#gas.TP = To, Po

In [3]:

To = 298
Po = 1e5 # ct.one_atm


gas = ct.Solution('./llnl/llnl/llnl.inp.cti') 
i = 0.025; mole_frac_list = []
while i < .5: 
    mole_frac_list.append(i) 
    i += 0.0025 
results = {}
results_final = {}

mole_frac_list

CanteraError: 
***********************************************************************
CanteraError thrown by Application::findInputFile:

Input file ./llnl/llnl/llnl.inp.cti not found in directories 
'.', 
'/home/khalil.nor/.conda/envs/cantera_env/lib/python3.9/site-packages/cantera/data', 
'/home/khalil.nor/.conda/envs/cantera_env/share/cantera/data'

To fix this problem, either:
    a) move the missing files into the local directory;
    b) define environment variable CANTERA_DATA to
         point to the directory containing the file.
***********************************************************************


In [ ]:
mole_frac_list = [0.01, 0.0566, 0.1]
# stoichiometric

In [ ]:

'''
mole_frac_dict = {'c2h6':x, 'O2':(1-x)*.21, 'N2':(1-x)*0.79 }
ratio = (mole_frac_dict['c2h6']/mole_frac_dict['O2'])/(1/3.5)
print(ratio)
gas.set_equivalence_ratio(0.041771094402673355, 'C2H6(1)','O2(2):0.21, N2:0.79') 
gas.TP = To, Po 
width = 0.08
flame = ct.FreeFlame(gas, width=width)
flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
loglevel = 1 
flame.solve(loglevel=loglevel, auto=True)
Su = flame.u[0]
results[x] = Su
sltn = flame.to_solution_array()
pd = sltn.to_pandas()
print(pd)
print(Su)

'''
for x in mole_frac_list: 
    mole_frac_dict = {'c2h6':x, 'O2':(1-x)*.21, 'N2':(1-x)*0.79 }
    ratio = (mole_frac_dict['c2h6']/mole_frac_dict['O2'])/(1/3.5)
    print(ratio)
    gas.set_equivalence_ratio(ratio, 'C2H6','O2:0.21, N2:0.79') 
    gas.TP = To, Po 
    width = 0.08
    flame = ct.FreeFlame(gas, width=width)
    flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
    loglevel = 1 
    flame.solve(loglevel=loglevel, auto=True)
    Su = flame.u[0]
    Su_final = flame.u[-1]
    results[x] = Su
    results_final[x] = Su_final
    sltn = flame.to_solution_array()
    pd = sltn.to_pandas()
    print(pd)



In [ ]:
flame.velocity

In [ ]:
pd.plot('grid')

In [ ]:
results

### Define flame simulation conditions

In [ ]:
#plotting 
xresults = list(results.keys())
yresults = list(results.values())

plt.figure()

plt.plot(xresults, yresults, '-o')
#plt.xlabel('Distance (cm)')
#plt.ylabel('Temperature (K)');

In [ ]:
# Domain width in metres
#width = 0.014

# Create the flame object
#flame = ct.FreeFlame(gas, width=width)

# Define tolerances for the solver
#flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1)

# Define logging level
#loglevel = 1

### Solve

In [ ]:
#flame.solve(loglevel=loglevel, auto=True)
#Su0 = flame.u[0]
print("Flame Speed is: {:.2f} cm/s".format(Su0*100))

# Note that the variable Su0 will also be used downsteam in the sensitivity analysis

In [ ]:
plt.figure()

plt.plot(flame.u, flame.T, '-o')
plt.xlabel('Distance (cm)')
plt.ylabel('Temperature (K)');

### Plot figures

Check and see if all has gone well. Plot temperature and species fractions to see

#### Temperature Plot

In [ ]:
plt.figure()

plt.plot(flame.grid*100, flame.T, '-o')
plt.xlabel('Distance (cm)')
plt.ylabel('Temperature (K)');

#### Major species' plot

To plot species, we first have to identify the index of the species in the array
For this, cut & paste the following lines and run in a new cell to get the index

    for i, specie in enumerate(gas.species()):
        print(str(i) + '. ' + str(specie))

In [ ]:
for i, specie in enumerate(gas.species()):
    print(str(i) + '. ' + str(specie))

In [ ]:
# Extract concentration data
X_C2H6 = flame.X[4]
X_CO2 = flame.X[48]


plt.figure()

plt.plot(flame.grid*100, X_CH4, '-o', label=r'$CH_{4}$')
plt.plot(flame.grid*100, X_CO2, '-s', label=r'$CO_{2}$')
plt.plot(flame.grid*100, X_H2O, '-<', label=r'$H_{2}O$')

plt.legend(loc=2)
plt.xlabel('Distance (cm)')
plt.ylabel('MoleFractions');

## Sensitivity Analysis

See which reactions effect the flame speed the most

In [ ]:
# Create a dataframe to store sensitivity-analysis data
sensitivities = pd.DataFrame(data=[], index=gas.reaction_equations(range(gas.n_reactions)))

### Compute sensitivities

In [ ]:
# Set the value of the perturbation
dk = 1e-2

# Create an empty column to store the sensitivities data
sensitivities["baseCase"] = ""

In [ ]:
for m in range(gas.n_reactions):
    gas.set_multiplier(1.0) # reset all multipliers                                                                     
    gas.set_multiplier(1+dk, m) # perturb reaction m   
    
    # Always force loglevel=0 for this
    # Make sure the grid is not refined, otherwise it won't strictly 
    # be a small perturbation analysis
    flame.solve(loglevel=0, refine_grid=False)
    
    # The new flame speed
    Su = flame.u[0]
    
    sensitivities["baseCase"][m] = (Su-Su0)/(Su0*dk)

# This step is essential, otherwise the mechanism will have been altered
gas.set_multiplier(1.0)

In [ ]:
sensitivities.head()

### Make plots

In [ ]:
# Reaction mechanisms can contains thousands of elementary steps. Choose a threshold
# to see only the top few
threshold = 0.03

firstColumn = sensitivities.columns[0]

# For plotting, collect only those steps that are above the threshold
# Otherwise, the y-axis gets crowded and illegible
sensitivitiesSubset = sensitivities[sensitivities[firstColumn].abs() > threshold]
indicesMeetingThreshold = sensitivitiesSubset[firstColumn].abs().sort_values(ascending=False).index
sensitivitiesSubset.loc[indicesMeetingThreshold].plot.barh(title="Sensitivities for GRI 3.0",
                                                          legend=None)
plt.gca().invert_yaxis()

plt.rcParams.update({'axes.labelsize': 20})
plt.xlabel(r'Sensitivity: $\frac{\partial\:\ln{S_{u}}}{\partial\:\ln{k}}$');

# Uncomment the following to save the plot. A higher than usual resolution (dpi) helps
# plt.savefig('sensitivityPlot', dpi=300)

In [ ]:
# Flame Speed with Sensitivity Analysis

In this example we simulate a freely-propagating, adiabatic, 1-D flame and
* Calculate its laminar burning velocity
* Perform a sensitivity analysis of its kinetics

The figure below illustrates the setup, in a flame-fixed co-ordinate system. The reactants enter with density $\rho_{u}$, temperature $T_{u}$ and speed $S_{u}$. The products exit the flame at speed $S_{b}$, density $\rho_{b}$ and temperature $T_{b}$.

<img src="images/flameSpeed.png" alt="Freely Propagating Flame" style="width: 300px;"/>

### Import Modules

from __future__ import print_function
from __future__ import division

import cantera as ct
import numpy as np

print("Running Cantera Version: " + str(ct.__version__))

# Import plotting modules and define plotting preference
%matplotlib notebook
import matplotlib.pylab as plt

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.figsize'] = (8,6)

# Get the best of both ggplot and seaborn
plt.style.use('ggplot')
plt.style.use('seaborn-deep')

plt.rcParams['figure.autolayout'] = True

# Import Pandas for DataFrames
import pandas as pd

### Define the reactant conditions, gas mixture and kinetic mechanism associated with the gas

#Inlet Temperature in Kelvin and Inlet Pressure in Pascals
#In this case we are setting the inlet T and P to room temperature conditions
To = 300
Po = 101325

#Define the gas-mixutre and kinetics
#In this case, we are choosing a GRI3.0 gas
gas = ct.Solution('gri30.cti')

# Create a stoichiometric CH4/Air premixed mixture 
gas.set_equivalence_ratio(1.0, 'CH4', {'O2':1.0, 'N2':3.76})
gas.TP = To, Po

### Define flame simulation conditions

# Domain width in metres
width = 0.014

# Create the flame object
flame = ct.FreeFlame(gas, width=width)

# Define tolerances for the solver
flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1)

# Define logging level
loglevel = 1

### Solve

flame.solve(loglevel=loglevel, auto=True)
Su0 = flame.u[0]
print("Flame Speed is: {:.2f} cm/s".format(Su0*100))

# Note that the variable Su0 will also be used downsteam in the sensitivity analysis

### Plot figures

Check and see if all has gone well. Plot temperature and species fractions to see

#### Temperature Plot

plt.figure()

plt.plot(flame.grid*100, flame.T, '-o')
plt.xlabel('Distance (cm)')
plt.ylabel('Temperature (K)');

#### Major species' plot

To plot species, we first have to identify the index of the species in the array
For this, cut & paste the following lines and run in a new cell to get the index

    for i, specie in enumerate(gas.species()):
        print(str(i) + '. ' + str(specie))

# Extract concentration data
X_CH4 = flame.X[13]
X_CO2 = flame.X[15]
X_H2O = flame.X[5]

plt.figure()

plt.plot(flame.grid*100, X_CH4, '-o', label=r'$CH_{4}$')
plt.plot(flame.grid*100, X_CO2, '-s', label=r'$CO_{2}$')
plt.plot(flame.grid*100, X_H2O, '-<', label=r'$H_{2}O$')

plt.legend(loc=2)
plt.xlabel('Distance (cm)')
plt.ylabel('MoleFractions');

## Sensitivity Analysis

See which reactions effect the flame speed the most

# Create a dataframe to store sensitivity-analysis data
sensitivities = pd.DataFrame(data=[], index=gas.reaction_equations(range(gas.n_reactions)))

### Compute sensitivities

# Set the value of the perturbation
dk = 1e-2

# Create an empty column to store the sensitivities data
sensitivities["baseCase"] = ""

for m in range(gas.n_reactions):
    gas.set_multiplier(1.0) # reset all multipliers                                                                     
    gas.set_multiplier(1+dk, m) # perturb reaction m   
    
    # Always force loglevel=0 for this
    # Make sure the grid is not refined, otherwise it won't strictly 
    # be a small perturbation analysis
    flame.solve(loglevel=0, refine_grid=False)
    
    # The new flame speed
    Su = flame.u[0]
    
    sensitivities["baseCase"][m] = (Su-Su0)/(Su0*dk)

# This step is essential, otherwise the mechanism will have been altered
gas.set_multiplier(1.0)

sensitivities.head()

### Make plots

# Reaction mechanisms can contains thousands of elementary steps. Choose a threshold
# to see only the top few
threshold = 0.03

firstColumn = sensitivities.columns[0]

# For plotting, collect only those steps that are above the threshold
# Otherwise, the y-axis gets crowded and illegible
sensitivitiesSubset = sensitivities[sensitivities[firstColumn].abs() > threshold]
indicesMeetingThreshold = sensitivitiesSubset[firstColumn].abs().sort_values(ascending=False).index
sensitivitiesSubset.loc[indicesMeetingThreshold].plot.barh(title="Sensitivities for GRI 3.0",
                                                          legend=None)
plt.gca().invert_yaxis()

plt.rcParams.update({'axes.labelsize': 20})
plt.xlabel(r'Sensitivity: $\frac{\partial\:\ln{S_{u}}}{\partial\:\ln{k}}$');

# Uncomment the following to save the plot. A higher than usual resolution (dpi) helps
# plt.savefig('sensitivityPlot', dpi=300)

In [ ]:
new_dict_forward = dict(zip(gas.reaction_equations(), gas.forward_rate_constants))
#print(new_dict_forward.items())
new_dict_reverse = dict(zip(gas.reaction_equations(), gas.reverse_rate_constants))
#print(new_dict_reverse.items())
new_dict_forward_sortedtuples = sorted(new_dict_forward.items(), key=lambda item: item[1], reverse=True)
new_dict_reverse_sortedtuples = sorted(new_dict_reverse.items(), key=lambda item: item[1], reverse=True)
#print(new_dict_forward_sortedtuples)
#print(new_dict_reverse_sortedtuples)
    